#### This is my Etsy Program that scrapes an Etsy CSV and cleans the data for easy analysis

###### Essentially, the CSV files provided by Etsy are messy and unusable without ample cleaning. 

###### This program automates that process by creating a list of the months orders and any revenues/costs associated with each.

###### This code does the following:

- Loads any csv files in the specified path.
- Cleans the data for analysis.
- Collects any costs/revenue associated with an order.
- Totals the costs and revenue and outputs a new CSV file with each order and the associated net revenue/cost.

###### An example of what this codes achieves:

###### Before (Total of 416 rows): https://imgur.com/a/KBgMOlr

###### After: https://imgur.com/a/f5nAvA0

In [1]:
# Importing necessary libraries
import pandas as pd
from datetime import datetime
import os
import glob

In [2]:
# Function to check if "Order" is present in a DataFrame row
def contains_order(row):
    return any("Order" in str(x) for x in row)

# Etsy_Scraper function to scrape data from CSV files and extract necessary details
def Etsy_Scraper():
    
    # Define the path to the directory where CSV files are located
    path = 'C:\\Users\\User\\Python\\Projects\\Etsy Scraper\\Etsy Data'
    output_path = 'C:\\Users\\User\\Python\\Projects\\Etsy Scraper\\Scraped Data'
    extension = 'csv'
    os.chdir(path)
    result = glob.glob('*.{}'.format(extension))

    # Loop over all CSV files in the directory
    for entry in result:
        # Only process files that contain 'etsy_statement' in their name
        if 'etsy_statement' in entry:
            # Read the CSV file into a DataFrame
            order_df = pd.read_csv(entry)

            # Select necessary columns and process the date
            order_df =  order_df[["Date", "Type", "Title", "Info", "Net"]]
            date = order_df['Date'].iloc[0]
            date = date.replace(',','')
            date = datetime.strptime(date, '%d %B %Y')
            month = date.month
            year = date.year

            # Create a DataFrame for sales and process it
            sale_df = order_df[["Date", "Type", "Title", "Net"]]
            sale_df = sale_df[sale_df.apply(contains_order, axis=1)]
            sale_df.rename(columns={'Title': 'order #'},inplace=True)
            sale_df['order #'] = sale_df['order #'].apply(lambda x: x[x.find('#'):])
            sale_df['Net'] = sale_df['Net'].apply(lambda x:x.replace(',',''))
            sale_df['Net'] = sale_df['Net'].apply(lambda x:float(x.replace('CA$','')))

            # Create a DataFrame for costs and process it
            cost_df = order_df[["Date", "Type", "Info", "Net"]]
            cost_df = cost_df[cost_df.apply(contains_order, axis=1)]
            cost_df.rename(columns={'Info': 'order #'},inplace=True)
            cost_df['order #'] = cost_df['order #'].apply(lambda x: x[x.find('#'):])
            cost_df['Net'] = cost_df['Net'].apply(lambda x:float(x.replace('CA$','')))

            # Merge the sales and costs DataFrames, group by 'order #' and sum 'Net'
            merged_df = pd.concat([sale_df, cost_df])
            merged_df = merged_df[['order #', 'Type', 'Net', 'Date']]
            merged_df = merged_df.groupby('order #').Net.sum().reset_index()

            # Save the merged DataFrame to a CSV file
            output_file = 'scraped_financials_{}_{}.csv'.format(year, month)
            output_file_path = os.path.join(output_path, output_file)
            merged_df.to_csv(output_file_path, index=False)

In [3]:
# Combine_Data function to combine all scraped CSV files into one
def Combine_Data():

    path = 'C:\\Users\\User\\Python\\Projects\\Etsy Scraper\\Scraped Data'
    output_path = 'C:\\Users\\User\\Python\\Projects\\Etsy Scraper\\Combined Data'
    extension = 'csv'
    os.chdir(path)
    result = glob.glob('*.{}'.format(extension))

    all_data = []

    # Loop over all CSV files in the directory
    for entry in result:
        # Only process files that contain 'scraped_financials' in their name
        if 'scraped_financials' in entry:
            # Read the CSV file into a DataFrame
            order_df = pd.read_csv(entry)
            order_df = order_df[["order #", "Net"]]

            # Append the DataFrame to the all_data list
            all_data.append(order_df)

    # Concatenate all DataFrames in the all_data list
    all_data = pd.concat(all_data, ignore_index=True)

    # Group by 'order #' and sum the 'Net' values
    all_data = all_data.groupby('order #', as_index=False).agg({'Net': 'sum'})

    # Save the combined DataFrame to a CSV file
    output_file = 'all_data.csv'
    output_file_path = os.path.join(output_path, output_file)
    all_data.to_csv(output_file_path, index=False)
    all_data.to_csv('concatenated_financials.csv', index=False)

In [4]:
# Run the Etsy_Scraper and Combine_Data functions
Etsy_Scraper()
Combine_Data()